In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
from itertools import product
from joblib import Parallel, delayed
from collections import Counter
from itertools import product
from collections import defaultdict
import time
from dask.diagnostics import ProgressBar
import dask.array as da
from dask import delayed, compute
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_parquet('../../data/processed/cov-19.parquet', engine='pyarrow')  # You can use 'fastparquet' as the engine
data

,Accession ID,Lineage,Sequence,Coverage,Train
0,EPI_ISL_15104785,BA.5.1,TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTC...,99.213260,2
1,EPI_ISL_3411570,AY.19,TGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTG...,99.939512,2
2,EPI_ISL_2433815,C.1,ATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGA...,95.359497,2
2,EPI_ISL_1715397,L.3,CTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGT...,99.775462,1
3,EPI_ISL_14795073,BA.4.6,ACCAGTGGCTTACCGCAAGGTTCTTCTTCGTAAGAACGGTAATAAA...,99.972625,0
...,...,...,...,...,...
46982,EPI_ISL_18407444,BA.4.1.8,TTGTAGATCTGTTCTCTAAACGNANNNNNNNNNNNNNNNNNNNNNN...,98.319865,1
46984,EPI_ISL_18407446,BA.4.1.8,TTGTAGATCTGTTCTCTAAACGAACNTGAAAANNNNNNNNNNNNNN...,99.107744,2
46985,EPI_ISL_18407448,BA.5.11,TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTC...,99.983160,2
46986,EPI_ISL_15426702,BA.4,TTGTAGATCTGTTCTCTAAACGAACNNNNNNNNNNNNNNNNNNNNN...,98.986805,2


In [3]:
def generate_kmers(k, alphabet='ACGT'):
    """Generate all possible k-mers from the given alphabet."""
    return [''.join(p) for p in product(alphabet, repeat=k)]

def find_spaced_words(sequence, pattern, pattern_indices):
    start = time.time()
    kmer_counts = defaultdict(int)  # Using defaultdict for faster updates
    all_kmers = generate_kmers(k)
    print(time.time() - start)
    for i in range(len(sequence) - len(pattern) + 1):
    # Preallocate list to avoid list comprehension overhead
        spaced_word_chars = [None] * len(pattern_indices)
        for index, j in enumerate(pattern_indices):
            spaced_word_chars[index] = sequence[i + j]
        spaced_word = ''.join(spaced_word_chars)
        kmer_counts[spaced_word] += 1
    
    # Only include k-mers present in all_kmers to match original function's behavior
    feature_vector = np.array([kmer_counts[kmer] for kmer in all_kmers])
    print(time.time() - start)
    return feature_vector

In [4]:
import random

def generate_random_pattern(k, l):
    one_positions = random.sample(range(2, l + k + 1), k - 1)
    one_positions.append(1)
    one_positions.sort()
    
    # Create the pattern, filling in '0's where there isn't a '1'
    pattern = ''
    for i in range(1, max(one_positions) + 1):
        if i in one_positions:
            pattern += '1'
        else:
            pattern += '0'
    
    return pattern

# Example usage
k = 7  # Number of '1's in the pattern
l = 30  # Length of the pattern
pattern = generate_random_pattern(k, l)
print(pattern)
pattern_indices = [i for i, char in enumerate(pattern) if char == '1']

100100001001000110000000000000000001


In [5]:
delayed_find_spaced_words = delayed(find_spaced_words)
tasks = [delayed_find_spaced_words(sequence, pattern, pattern_indices) for sequence in data["Sequence"]]

# Instantiate and register the ProgressBar
progress_bar = ProgressBar()
progress_bar.register()

# Use dask.compute to execute tasks in parallel, now with a progress bar
with progress_bar:
    feature_vectors = compute(*tasks, scheduler='processes')

# Stack the resulting feature vectors
stacked_feature_vectors = np.vstack(feature_vectors)

[#                                       ] | 4% Completed | 7.37 s ms0.05212593078613281
0.3635890483856201
0.018660545349121094
0.12949848175048828
0.0033292770385742188
0.17565393447875977
0.002540111541748047
0.03622269630432129
0.0016164779663085938
0.027685880661010742
0.0015420913696289062
0.026998043060302734
0.002966165542602539
0.03506112098693848
0.0015845298767089844
0.029251575469970703
0.0015435218811035156
0.031394243240356445
0.0015864372253417969
0.02711629867553711
0.001638650894165039
0.031104564666748047
0.0015633106231689453
0.027071714401245117
0.002801179885864258
0.031453847885131836
0.002576589584350586
0.03066706657409668
0.0015628337860107422
0.027333974838256836
0.0016019344329833984
0.02784562110900879
0.001592874526977539
0.028486013412475586
0.0015621185302734375
0.027280092239379883
0.0015807151794433594
0.029947996139526367
0.0015184879302978516
0.02937769889831543
0.0015125274658203125
0.027627944946289062
0.001508951187133789
0.029248952865600586
0.001

Process SpawnProcess-1:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/opt/conda/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/opt/conda/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.9/multiprocessing/connection.py", line 384, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
spaced_words = pd.DataFrame(stacked_feature_vectors)

In [7]:
spaced_words["Target"] = data["Lineage"].tolist()
spaced_words["Train"] = data["Train"].tolist()

In [8]:
spaced_words.columns = spaced_words.columns.astype(str)

In [9]:
spaced_words.to_parquet(f'../../data/features/7-spaced.parquet', engine='pyarrow')